In [39]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import time
import os
import pandas as pd
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

from tqdm import tqdm

In [40]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


In [41]:
driver_path = '/Users/josep/Documents/chromedriver-mac-x64/chromedriver'

In [42]:
def scroll_page(scrolls_num):
    for _ in range(scrolls_num):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

In [55]:
def get_detailed_info():
    
    # Reading price
    price = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//span[@class='re-DetailHeader-price']"))
    ).text

    try:
        discount = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='re-DetailHeader-reducedPrice']"))
        ).text
    except:
        discount = 'No hay descuento'
    
    # Reading basic Features
    basic_features = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//ul[@class='re-DetailHeader-features']"))
    )
    basic_features_list = basic_features.text.split('\n')
    features = {f"feature{i}": basic_features_list[i] for i in range(0, len(basic_features_list))}

    # Reading title and subtitle
    title = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//h1[@class='re-DetailHeader-propertyTitle']"))
    ).text
    subtitle = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//p[@class='re-DetailHeader-municipalityTitle']"))
    ).text
    
    # Reading description (click read more)
    description = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//p[@class='fc-DetailDescription']"))
    ).text
    
    # Characteristics
    names = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//p[@class='re-DetailFeaturesList-featureLabel']"))
    )
    names_list = [n.text for n in names]
    characteristics = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//p[@class='re-DetailFeaturesList-featureValue']"))
    )
    characteristics_list = [c.text for c in characteristics]
    characteristics = {name:c for name, c in zip(names_list, characteristics_list)}
    
    # Extra characteristics
    extras = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//ul[@class='re-DetailExtras-list']"))
    )
    extras_list = extras.text.split("\n")
    
    # Address
    # driver.execute_script("arguments[0].scrollIntoView(true);", extras)
    time.sleep(1)
    address = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//h2[@class='re-DetailMap-address']"))
    ).text

    return price, discount, features, title, subtitle, description, characteristics, extras_list, address



In [56]:
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
user_agent = UserAgent()
options.add_argument(f'user-agent={user_agent.random}')
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.fotocasa.es/")

# Deny cookies button
button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//button[@id='didomi-notice-agree-button']"))
)
button.click()

# Detect and write in placeholder
buscador = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@class='sui-AtomInput-input sui-AtomInput-input-size-m']"))
)
buscador.send_keys("Barcelona")

# Click Barcelona Capital 
button_bcn = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//li[@data-value='Barcelona Capital, Barcelona']"))
)
button_bcn.click()

time.sleep(3)



In [58]:
page_prices = []
page_discounts = []
page_features = []
page_titles = []
page_subtitles = []
page_descriptions = []
page_characteristics = []
page_extras = []
page_addresses = []


for i in tqdm(range(0,30)):
    scroll_page(i+6)
    
    flat_list_page = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//section[@class='re-SearchResult']//article"))
    )
    
    flat_list_page[i].click()
    # ActionChains(driver).key_down(Keys.COMMAND).click(flat_list_page[i]).key_up(Keys.COMMAND).perform()
    # driver.switch_to.window(driver.window_handles[-1])
    
    time.sleep(1)
    
    scroll_page(3)

    price, discount, features, title, subtitle, description, characteristics, extras_list, address = get_detailed_info()
    page_prices.append(price)
    page_discounts.append(discount)
    page_features.append(features)
    page_titles.append(title)
    page_subtitles.append(subtitle)
    page_descriptions.append(description)
    page_characteristics.append(characteristics)
    page_extras.append(extras_list)
    page_addresses.append(address)
    
    driver.back()

    time.sleep(1)

    driver.execute_script("window.scrollTo(0, 0);")

    time.sleep(1)
    

 57%|████████████████████████▎                  | 17/30 [07:40<05:51, 27.07s/it]


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000102605168 chromedriver + 4673896
1   chromedriver                        0x00000001025fc9c3 chromedriver + 4639171
2   chromedriver                        0x00000001021f0fdd chromedriver + 397277
3   chromedriver                        0x000000010223cbfc chromedriver + 707580
4   chromedriver                        0x000000010223cdd1 chromedriver + 708049
5   chromedriver                        0x0000000102281284 chromedriver + 987780
6   chromedriver                        0x000000010225f8ed chromedriver + 850157
7   chromedriver                        0x000000010227e796 chromedriver + 976790
8   chromedriver                        0x000000010225f663 chromedriver + 849507
9   chromedriver                        0x000000010222f1cf chromedriver + 651727
10  chromedriver                        0x00000001022301ae chromedriver + 655790
11  chromedriver                        0x00000001025c5380 chromedr

In [282]:
html_txt = driver.page_source
soup = BeautifulSoup(html_txt)
titles = soup.find_all("span",class_="re-CardTitle re-CardTitle--big")
titles_list = [title.getText() for title in titles]
agencies = soup.find_all("span",class_="re-CardTitle re-CardTitle--big")
agencies_list = [agency.getText() for agency in agencies]


In [269]:
"""html_txt = driver.page_source
soup = BeautifulSoup(html_txt)
price = soup.find("span", class_="re-DetailHeader-price").getText()
try: discount = soup.find("div", class_="re-DetailHeader-reducedPrice").getText()
except: discount = 0
basic_features = soup.find_all("li", class_="re-DetailHeader-featuresItem")
soup_features = BeautifulSoup(str(basic_features[0]))
soup_features.find("span", class_="").getText()"""

'html_txt = driver.page_source\nsoup = BeautifulSoup(html_txt)\nprice = soup.find("span", class_="re-DetailHeader-price").getText()\ntry: discount = soup.find("div", class_="re-DetailHeader-reducedPrice").getText()\nexcept: discount = 0\nbasic_features = soup.find_all("li", class_="re-DetailHeader-featuresItem")\nsoup_features = BeautifulSoup(str(basic_features[0]))\nsoup_features.find("span", class_="").getText()'